### I. Library import 


In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive,HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data2 = pd.read_csv("data.csv",low_memory=False)
data1 = pd.read_csv("data2023.csv",low_memory=False)
data = pd.concat([data1, data2], ignore_index=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599953 entries, 0 to 1599952
Data columns (total 10 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   date_mutation              1599953 non-null  object 
 1   nature_mutation            1599953 non-null  object 
 2   valeur_fonciere            1599953 non-null  float64
 3   code_commune               1599953 non-null  object 
 4   code_departement           1599953 non-null  object 
 5   id_parcelle                1599953 non-null  object 
 6   nombre_lots                1599953 non-null  int64  
 7   code_type_local            1599953 non-null  float64
 8   nombre_pieces_principales  1599953 non-null  float64
 9   surface_terrain            1599953 non-null  float64
dtypes: float64(4), int64(1), object(5)
memory usage: 122.1+ MB


### II. Création des fonctions

In [5]:
def afficher_prix_moyen_par_m2(df):
    # Dictionnaire de mappage pour les types de bien
    type_local_map = {
        1: 'Maison',
        2: 'Appartement',
        3: 'Dépendance (isolée)',
        4: 'Local industriel et commercial ou assimilés'
    }

    # Remplacer les codes de type_local par les descriptions lisibles
    df['type_local'] = df['code_type_local'].map(type_local_map)

    # Liste des départements uniques
    departements = df['code_departement'].unique()

    # Liste des types de biens uniques
    type_bien = df['type_local'].unique()

    # Sélecteur de départements
    departement_select = widgets.SelectMultiple(
        options=departements,
        value=[],
        description='Départements',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    departement_select.style.description_width = '100px'

    # Sélecteur de types de biens
    type_bien_select = widgets.SelectMultiple(
        options=type_bien,
        value=[],
        description='Type de bien',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    type_bien_select.style.description_width = '100px'

    def plot_prix_moyen_par_m2(_=None):
        # Récupérer les départements et types de bien sélectionnés
        selected_departements = list(departement_select.value)
        selected_types_bien = list(type_bien_select.value)

        # Si rien n'est sélectionné, on prend tous les départements et types de biens
        if not selected_departements:
            selected_departements = list(df['code_departement'].unique())
        if not selected_types_bien:
            selected_types_bien = list(df['type_local'].unique())

        # Filtrer les données en fonction des départements et types de bien sélectionnés
        df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                         (df['type_local'].isin(selected_types_bien))].copy()

        # Calculer le prix moyen par m² par département
        df_filtered['prix_m2'] = df_filtered['valeur_fonciere'] / df_filtered['surface_terrain']
        prix_moyen_par_dept = df_filtered.groupby('code_departement')['prix_m2'].mean()

        # Calculer la moyenne des prix des départements sélectionnés
        prix_moyen_selectionne = prix_moyen_par_dept.mean()

        # Calculer la moyenne nationale des prix par m²
        prix_moyen_national = df[df['type_local'].isin(selected_types_bien)].groupby('code_departement')['prix_m2'].mean().mean()

        # Création du graphique
        fig, ax = plt.subplots(figsize=(20, 6))

        # Tracer le prix moyen pour chaque département sélectionné
        ax.bar(prix_moyen_par_dept.index, prix_moyen_par_dept)

        # Ajouter une barre pour la moyenne des départements sélectionnés
        ax.bar('Moyenne Sélectionnée', prix_moyen_selectionne, alpha=0.7)

        # Ajouter une barre pour la moyenne nationale
        ax.bar('Moyenne Nationale', prix_moyen_national, alpha=0.7)

        # Paramètres du graphique
        ax.set_ylabel('Prix moyen par m² (€)')
        ax.set_xlabel('Départements')
        ax.set_title("Prix moyen par m² par département, sélection et moyenne globale")
        ax.grid(True, linestyle='--', alpha=0.6)
        plt.xticks(rotation=90)
        plt.tight_layout()  # Pour éviter que les labels se chevauchent
        plt.show()

    # Bouton pour afficher les résultats
    button = widgets.Button(description="Afficher les résultats", button_style='info')
    button.on_click(plot_prix_moyen_par_m2)

    # Affichage des widgets pour la sélection
    display(departement_select)
    display(type_bien_select)
    display(button)

In [6]:
def afficher_surface_moyenne_par_dept(df):
    # Dictionnaire de mappage pour les types de bien
    type_local_map = {
        1: 'Maison',
        2: 'Appartement',
        3: 'Dépendance (isolée)',
        4: 'Local industriel et commercial ou assimilés'
    }

    # Remplacer les codes de type_local par les descriptions lisibles
    df['type_local'] = df['code_type_local'].map(type_local_map)

    # Liste des départements uniques
    departements = df['code_departement'].unique()

    # Liste des types de biens uniques
    type_bien = df['type_local'].unique()

    # Sélecteur de départements
    departement_select = widgets.SelectMultiple(
        options=departements,
        value=[],
        description='Départements',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    departement_select.style.description_width = '100px'

    # Sélecteur de types de biens
    type_bien_select = widgets.SelectMultiple(
        options=type_bien,
        value=[],
        description='Type de bien',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    type_bien_select.style.description_width = '100px'

    def plot_surface_moyenne_par_dept(_=None):
        # Récupérer les départements et types de bien sélectionnés
        selected_departements = list(departement_select.value)
        selected_types_bien = list(type_bien_select.value)

        # Si rien n'est sélectionné, on prend tous les départements et types de biens
        if not selected_departements:
            selected_departements = list(df['code_departement'].unique())
        if not selected_types_bien:
            selected_types_bien = list(df['type_local'].unique())

        # Filtrer les données en fonction des départements et types de bien sélectionnés
        df_filtered = df[(df['code_departement'].isin(selected_departements)) & 
                         (df['type_local'].isin(selected_types_bien))].copy()

        
        # Calculer la surface moyenne pour chaque département sélectionné
        surface_moyenne_par_dept = df_filtered.groupby('code_departement')['surface_terrain'].mean()

        # Calculer la moyenne des surfaces des départements sélectionnés
        surface_moyenne_selectionnee = surface_moyenne_par_dept.mean()

        # Calculer la surface moyenne nationale (moyenne de tous les départements et types de bien sélectionnés)
        surface_moyenne_nationale = df[df['type_local'].isin(selected_types_bien)].groupby('code_departement')['surface_terrain'].mean().mean()

        # Création du graphique
        fig, ax = plt.subplots(figsize=(20, 6))

        # Tracer la surface moyenne pour chaque département sélectionné
        ax.bar(surface_moyenne_par_dept.index, surface_moyenne_par_dept, label='Surface Moyenne par Département')

        # Ajouter une barre pour la moyenne des départements sélectionnés
        ax.bar('Moyenne Sélectionnée', surface_moyenne_selectionnee, label='Moyenne Sélectionnée', alpha=0.7)

        # Ajouter une barre pour la moyenne nationale
        ax.bar('Moyenne Nationale', surface_moyenne_nationale, label='Moyenne Nationale', alpha=0.7)

        # Paramètres du graphique
        ax.set_ylabel('Surface moyenne en m²')
        ax.set_xlabel('Départements')
        ax.set_title("Surface moyenne en m² par département, sélection et moyenne globale")
        ax.grid(True, linestyle='--', alpha=0.6)
        ax.legend(title="Légende", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=90)
        plt.tight_layout()  # Pour éviter que les labels se chevauchent
        plt.show()

    # Bouton pour afficher les résultats
    button = widgets.Button(description="Afficher les résultats", button_style='info')
    button.on_click(plot_surface_moyenne_par_dept)

    # Affichage des widgets pour la sélection
    display(departement_select)
    display(type_bien_select)
    display(button)


In [11]:
def evolution_prix_moyen_par_m2_par_departement_mensuel(df):
    type_local_map = {
        1: 'Maison',
        2: 'Appartement',
        3: 'Dépendance (isolée)',
        4: 'Local industriel et commercial ou assimilés'
    }

    # Remplacer les codes de type_local par les descriptions lisibles
    df['type_local'] = df['code_type_local'].map(type_local_map)

    departements = df['code_departement'].unique()

    departement_select = widgets.SelectMultiple(
        options=departements,
        value=[],
        description='Départements',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    departement_select.style.description_width = '100px'

    type_maison = df['type_local'].unique()

    type_bien_select = widgets.SelectMultiple(
        options=type_maison,
        value=[],
        description='Type de bien',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    type_bien_select.style.description_width = '100px'

    # Fonction pour afficher l'évolution du prix moyen par m² mensuel
    def afficher_graphique(_=None):
        selected_departements = list(departement_select.value)
        selected_types_bien = list(type_bien_select.value)

        if not selected_departements:
            selected_departements = list(df['code_departement'].unique())
        if not selected_types_bien:
            selected_types_bien = list(df['type_local'].unique())

        # Filtrer les données en fonction des départements et types de bien sélectionnés
        df_filtered = df[df['code_departement'].isin(selected_departements) & df['type_local'].isin(selected_types_bien)].copy()

        # S'assurer que la colonne de date est au bon format (si ce n'est pas déjà le cas)
        df_filtered['date_mutation'] = pd.to_datetime(df_filtered['date_mutation'])

        # Calculer le prix au m²
        df_filtered['prix_m2'] = df_filtered['valeur_fonciere'] / df_filtered['surface_terrain']

        # Extraire l'année et le mois de la date de transaction
        df_filtered['mois_annee'] = df_filtered['date_mutation'].dt.to_period('M')

        # Groupement par département et par mois
        prix_moyen_par_dept_mois = df_filtered.groupby(['code_departement', 'mois_annee'])['prix_m2'].mean().reset_index()

        # Créer le graphique
        plt.figure(figsize=(15, 8))

        # Ajouter la courbe pour la moyenne générale
        df_dept_global = df_filtered.groupby('mois_annee')['prix_m2'].mean().reset_index()
        plt.plot(df_dept_global['mois_annee'].astype(str), df_dept_global['prix_m2'], marker='o', label='Moyenne générale', linewidth=2)

        # Ajouter la courbe pour la moyenne nationale (basée sur l'intégralité des données)
        df_national_global = df_filtered.groupby('mois_annee')['prix_m2'].mean().reset_index()
        plt.plot(df_national_global['mois_annee'].astype(str), df_national_global['prix_m2'], marker='o', label='Moyenne nationale', linewidth=2)

        # Ajouter les courbes pour chaque département sélectionné
        for departement in selected_departements:
            df_dept = prix_moyen_par_dept_mois[prix_moyen_par_dept_mois['code_departement'] == departement]
            plt.plot(df_dept['mois_annee'].astype(str), df_dept['prix_m2'], marker='o', label=f'Département {departement}')

        plt.title("Évolution du prix moyen par m² par département (mensuel)")
        plt.xlabel("Mois et Année")
        plt.ylabel("Prix moyen par m² (€)")
        plt.legend(title="Types de Bien", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.xticks(rotation=45)
        plt.tight_layout()  # Pour éviter que les labels se chevauchent
        plt.show()

    # Bouton pour afficher les résultats
    button = widgets.Button(description="Afficher les résultats", button_style='info')
    button.on_click(afficher_graphique)

    # Afficher les widgets pour la sélection
    display(departement_select)
    display(type_bien_select)
    display(button)


In [19]:
def afficher_nombre_piece_departement(df):
    # Dictionnaire de mappage pour les types de bien
    type_local_map = {
        1: 'Maison',
        2: 'Appartement',
        3: 'Dépendance (isolée)',
        4: 'Local industriel et commercial ou assimilés'
    }

    # Remplacer les codes de type_local par les descriptions lisibles
    df['type_local'] = df['code_type_local'].map(type_local_map)

    departements = df['code_departement'].unique()

    # Widget pour sélectionner les départements
    departement_select = widgets.SelectMultiple(
        options=departements,
        value=[],
        description='Départements',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    departement_select.style.description_width = '100px'

    # Widget pour sélectionner les types de bien
    type_maison = df['type_local'].unique()

    type_bien_select = widgets.SelectMultiple(
        options=type_maison,
        value=[],
        description='Type de bien',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    type_bien_select.style.description_width = '100px'

    # Fonction pour afficher le graphique
    def afficher_graphique(_=None):
        selected_departements = list(departement_select.value)
        selected_types_bien = list(type_bien_select.value)

        # Si aucun département n'est sélectionné, afficher tous les départements
        if not selected_departements:
            selected_departements = list(df['code_departement'].unique())
        
        # Si aucun type de bien n'est sélectionné, afficher tous les types de bien
        if not selected_types_bien:
            selected_types_bien = list(df['type_local'].unique())

        # Filtrer les données en fonction des départements et types de bien sélectionnés
        df_filtered = df[df['code_departement'].isin(selected_departements) & df['type_local'].isin(selected_types_bien)].copy()

        # Calculer le nombre moyen de pièces
        df_filtered['nombre_piece'] = df_filtered['nombre_pieces_principales']

        # Groupement par département pour calculer la moyenne des pièces
        nombre_piece_par_dept = df_filtered.groupby('code_departement')['nombre_piece'].mean().reset_index()

        # Ajouter la moyenne générale des départements sélectionnés
        moyenne_selection = nombre_piece_par_dept['nombre_piece'].mean()

        # Ajouter une barre pour la moyenne générale
        departements = list(nombre_piece_par_dept['code_departement'])
        nombre_piece = list(nombre_piece_par_dept['nombre_piece'])

        # Créer le graphique
        plt.figure(figsize=(15, 8))
        plt.bar(departements, nombre_piece, color='skyblue')
        plt.axhline(moyenne_selection, color='red', linestyle='dashed', linewidth=2, label='Moyenne Sélectionnée')

        plt.title("Nombre moyen de pièces par département et par type de bien sélectionné")
        plt.xlabel("Département")
        plt.ylabel("Nombre moyen de pièces")
        plt.xticks(rotation=90)
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()  # Pour éviter que les labels se chevauchent
        plt.show()

    # Bouton pour afficher les résultats
    button = widgets.Button(description="Afficher les résultats", button_style='info')
    button.on_click(afficher_graphique)

    # Afficher les widgets pour la sélection
    display(departement_select)
    display(type_bien_select)
    display(button)


In [14]:
def repartition_type_bien_par_departement(df):
    # Dictionnaire de la liste de type de bien
    type_local_map = {
        1: 'Maison',
        2: 'Appartement',
        3: 'Dépendance (isolée)',
        4: 'Local industriel et commercial ou assimilés'
    }
    df['type_local'] = df['code_type_local'].map(type_local_map)

    # creation du widget du choix de département
    departements = df['code_departement'].unique()
    departement_select = widgets.SelectMultiple(
        options=departements,
        value=[],
        description='Départements',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    departement_select.style.description_width = '100px'

    # Fonction pour afficher la répartition des types de bien par département
    def afficher_graphique(_=None):
        selected_departements = list(departement_select.value)

        if not selected_departements:
            selected_departements = list(df['code_departement'].unique())

        # Filtrer les données pour les départements sélectionnés
        df_filtered = df[df['code_departement'].isin(selected_departements)].copy()

        # Calculer la répartition des types de biens par département
        repartition = df_filtered.groupby(['code_departement', 'type_local']).size().unstack(fill_value=0)

        # Normaliser pour obtenir des proportions (en %)
        repartition_norm = repartition.div(repartition.sum(axis=1), axis=0) * 100

        # Création d'un graphique en barres empilées
        repartition_norm.plot(kind='bar', figsize=(15, 8), stacked=True, colormap='tab20')

        #  option d'affichage du graphique
        plt.title("Répartition des transactions par type de bien et par département")
        plt.xlabel("Département")
        plt.ylabel("Proportion des transactions (%)")
        plt.legend(title="Type de bien", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        plt.xticks(rotation=45)
        plt.tight_layout()

        plt.show()

    # Bouton pour afficher le graphique 
    button = widgets.Button(description="Afficher les résultats", button_style='info')
    button.on_click(afficher_graphique)

    # Affichage des widget selectionnables
    display(departement_select)
    display(button)


In [15]:
def repartition_Valeur_fonciere_par_tranche(df):
    # Tranches de valeurs foncière
    tranches_valeur = [0, 50000, 100000, 200000, 300000, 500000, 1000000, float('inf')]
    labels_tranches = [
        "< 50k €",
        "50k - 100k €",
        "100k - 200k €",
        "200k - 300k €",
        "300k - 500k €",
        "500k - 1M €",
        "> 1M €"
    ]

    # Ajouter une colonne pour la tranche de valeur foncière
    df['tranche_valeur'] = pd.cut(df['valeur_fonciere'], bins=tranches_valeur, labels=labels_tranches, include_lowest=True)

    # creation du widget du choix de département
    departements = df['code_departement'].unique()
    departement_select = widgets.SelectMultiple(
        options=departements,
        value=[],
        description='Départements',
        disabled=False,
        layout=widgets.Layout(width='60%')
    )
    departement_select.style.description_width = '100px'

    # Fonction pour afficher la répartition des transactions
    def afficher_graphique(_=None):
        selected_departements = list(departement_select.value)

        if not selected_departements:
            selected_departements = list(df['code_departement'].unique())

        # Filtrer les données en fonction des départements sélectionnés
        df_filtered = df[df['code_departement'].isin(selected_departements)].copy()

        # Compter le nombre de transactions par tranche et par département
        repartition = df_filtered.groupby(['code_departement', 'tranche_valeur']).size().unstack(fill_value=0)

        # Normaliser pour obtenir des proportions (en %)
        repartition_norm = repartition.div(repartition.sum(axis=1), axis=0) * 100

        # Création du graphique
        repartition_norm.T.plot(kind='bar', figsize=(15, 8), stacked=True, colormap='tab10')

        # option d'affichage du graphique
        plt.title("Répartition des transactions par tranche de valeur foncière et par département")
        plt.xlabel("Tranche de valeur foncière (€)")
        plt.ylabel("Proportion des transactions (%)")
        plt.legend(title="Département", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        plt.xticks(rotation=45)
        plt.tight_layout()

        plt.show()

    # Bouton pour afficher le graphique 
    button = widgets.Button(description="Afficher les résultats", button_style='info')
    button.on_click(afficher_graphique)

    # Affichage des widget selectionnables
    display(departement_select)
    display(button)

### III. Exécution des fonctions

In [8]:
afficher_prix_moyen_par_m2(data)

SelectMultiple(description='Départements', layout=Layout(width='60%'), options=('01', '02', '03', '04', '05', …

SelectMultiple(description='Type de bien', layout=Layout(width='60%'), options=('Maison', 'Dépendance (isolée)…

Button(button_style='info', description='Afficher les résultats', style=ButtonStyle())

In [13]:
evolution_prix_moyen_par_m2_par_departement_mensuel(data)

SelectMultiple(description='Départements', layout=Layout(width='60%'), options=('01', '02', '03', '04', '05', …

SelectMultiple(description='Type de bien', layout=Layout(width='60%'), options=('Maison', 'Dépendance (isolée)…

Button(button_style='info', description='Afficher les résultats', style=ButtonStyle())

In [18]:
afficher_surface_moyenne_par_dept(data)

SelectMultiple(description='Départements', layout=Layout(width='60%'), options=('01', '02', '03', '04', '05', …

SelectMultiple(description='Type de bien', layout=Layout(width='60%'), options=('Maison', 'Dépendance (isolée)…

Button(button_style='info', description='Afficher les résultats', style=ButtonStyle())

In [20]:
afficher_nombre_piece_departement(data)

SelectMultiple(description='Départements', layout=Layout(width='60%'), options=('01', '02', '03', '04', '05', …

SelectMultiple(description='Type de bien', layout=Layout(width='60%'), options=('Maison', 'Dépendance (isolée)…

Button(button_style='info', description='Afficher les résultats', style=ButtonStyle())

In [17]:
repartition_type_bien_par_departement(data)

SelectMultiple(description='Départements', layout=Layout(width='60%'), options=('01', '02', '03', '04', '05', …

Button(button_style='info', description='Afficher les résultats', style=ButtonStyle())

In [16]:
repartition_Valeur_fonciere_par_tranche(data)

SelectMultiple(description='Départements', layout=Layout(width='60%'), options=('01', '02', '03', '04', '05', …

Button(button_style='info', description='Afficher les résultats', style=ButtonStyle())